In [0]:
import glob

# Importing drive method from colab for accessing google drive
from google.colab import drive

In [0]:
!ls

df_matched	df_not_matched.csv  fashion_stringmatch_dnn.csv
df_matched.csv	drive		    sample_data


In [0]:
# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/"


 01d372957654dad05db5fa36451b5854.jpg   8825d01d1e5f9e4a0335b5cabe66c9fa.jpg
 064a7ac32cecb53767d4c53924ad360e.jpg   8d08f1098529cedd36ef9dea3c766f08.jpg
 0c392b5acdb4ae7fc6078f21cc882296.jpg   92754481a52cfe5b928166699b32e340.jpg
 0cbd53a84d01a8019f1b7ef925637c68.jpg   94dd1c85a2b4c4d5dd50a0c1e2752f02.jpg
 17fab75058dfaa866747de6a8380a0fa.jpg   9664112f39f8ee5dfade79a23fb863ac.jpg
 25c6a756eab431cd7e5e7a96acb48d9e.jpg   9aa9ca9780aed7d070917e9c4a17df91.jpg
 2c3103564b75cbed741cb995cc60e620.jpg   9fe2867e00bf591aa4f57014611baaa1.jpg
 30daf99b481bf59fce449411c30dbd88.jpg   a8a4db6aa5f3aa8d071434dabcb85f87.jpg
 32e6a22d6cf7b4c95ace4c45e9f21fe2.jpg   add4e5dbbffed1da5c8aeb4a88ab1386.jpg
 3432c4ece2ca7e9fa81ce1dd1788f231.jpg   b2232d4db391a469db24608baad13bf5.jpg
 37305459e6e58e30cdf9993879c9aa7e.jpg   b401464a6ea7c5a3aba2cc2d11421bc1.jpg
 3d47ff1a7ffda5e8c15f09ecbcd79df8.jpg   b56b8bacc1bc8549c7845ee25b6eaca5.jpg
 3fba8cc63bddc24ad70de51dfd66f5c5.jpg   bcd5940512f0a3c696e2040d9afa1331.jpg

## Split the df

In [0]:
import pandas as pd

testset = pd.read_csv("/content/drive/My Drive/NDSC/data/test.csv")
df_test = fashion_test = testset.iloc[76545:131985]

df_test_predict = df_test
df_test.head()



,itemid,title,image_path
76545,670968580,women casual o neck long sleeve lace patchwork...,fashion_image/b0ca9645c5c3527882d935f17b4c9579...
76546,475599610,zaful long dress vintage wanita v neck lengan ...,fashion_image/c74af2636e9fe2686981decce3ca5b6c...
76547,1652671018,dress skater swing o neck tanpa lengan desain ...,fashion_image/1f2a2d91f5eb2d80f1dfdcf0b1f50169...
76548,1705634904,ok dress slim sexy deep v neck lengan panjang ...,fashion_image/fdb6309635ebb5eac5df9c61984ffce5...
76549,1789421087,noa korean lace embroidery womens crew neck lo...,fashion_image/578e5c74e97128cd25378ef0f0c021cf...


In [0]:
import json

with open('/content/drive/My Drive/NDSC/data/2_fashion_categories.json') as f:
    fashion_categories_json = json.load(f)

# process category.json into another dictionary suitable for the string matching library
fashion_category_titles = {}
for fashion_category_class in fashion_categories_json:
    for fashion_category_name in fashion_categories_json[fashion_category_class]:
        fashion_category_titles[fashion_categories_json[fashion_category_class][fashion_category_name]] = fashion_category_name
print(fashion_category_titles)  

# keywords_df=pd.read_csv("/content/drive/My Drive/NDSC/fashion_keywords.csv")
# list_keywords = keywords_df['keywords'].values.tolist()



{23: 'Wedding Dress', -23: 'bride', 18: 'Casual Dress', 20: 'Maxi Dress', 24: 'Big Size Dress', 22: 'Bodycon', 19: 'Party Dress', 26: 'Blouse', -26: 'blus', 29: 'Crop Top ', 28: 'Tanktop', 21: 'Line Dress', 30: 'Big Size Top', -30: 'xl'}


In [0]:
!pip install fuzzywuzzy

In [0]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

from tqdm import tqdm  # progress bar

In [0]:

import keras
from keras.models import Sequential, model_from_json
from keras.models import Model
from keras.layers import InputLayer, Input
from keras.layers import Reshape, MaxPooling2D
#from tensorflow.python.keras.layers import Conv2D, Dense, Flatten, Concatenate, concatenate, Dropout
from keras.layers import concatenate, Concatenate, Conv2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator, image

json_file = open('/content/drive/My Drive/models/75_fashion.json','r')
loaded_model_json = json_file.read()
json_file.close()
fashion_text_model = model_from_json(loaded_model_json)
fashion_text_model.load_weights("/content/drive/My Drive/models/75_fashion.h5")
print("Loaded model from disk")
# fashion_text_model = Model(fashion_text_model.input, last_layer)
fashion_text_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
fashion_text_model.summary()

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_6 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)             

In [0]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from tqdm import tqdm  # progress bar

In [0]:


# fuzzy string matching
# https://github.com/seatgeek/fuzzywuzzy
# on how to return index instead of the value
# https://github.com/seatgeek/fuzzywuzzy/issues/165
# prediction = []

df_matched = pd.DataFrame(columns=['itemid', 'title', 'category'])
df_not_matched = pd.DataFrame(columns=['itemid', 'title'])
threshold=100


for index, row in df_test.iterrows():
  itemid=row['itemid']
  title=row['title']
  low=[]
  
  predicted = process.extract(title, fashion_category_titles, scorer=fuzz.partial_ratio)

  for item in predicted:
    if item[1]<threshold:
      low.append(item)

  for item in low:
    predicted.remove(item)
    
    
  if len(predicted) ==1:
    predicted_cat=predicted[0][2]
    data = [itemid,title,predicted_cat]
    df2=pd.DataFrame(columns=['itemid', 'title', 'category'], data=[data])
    df2['itemid']=itemid
    df2['title']=title
    df2['category']=predicted_cat
    df_matched = df_matched.append(df2, ignore_index=True)
  
  else:
    data = [itemid,title]
    df2=pd.DataFrame(columns=['itemid', 'title'], data=[data])
    df_not_matched = df_not_matched.append(df2, ignore_index=True)
    

   

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [0]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils


from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

In [0]:
unmatched_test_title = df_not_matched['title']
max_words = 1000
batch_size = 500
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

train = pd.read_csv('/content/drive/My Drive/NDSC/data/train.csv')
fashion_train = train.iloc[286583:506285]
train_title = fashion_train['title']
tokenize.fit_on_texts(train_title) # only fit on train

In [0]:
x_test = tokenize.texts_to_matrix(unmatched_test_title)
unmatched_test_predict = fashion_text_model.predict(x_test, batch_size=batch_size, verbose=1)
unmatched_test_predict = pd.DataFrame(data=unmatched_test_predict)
unmatched_test_predict = unmatched_test_predict.idxmax(axis=1)



36391/36391 [==============================] - 0s 11us/step


In [0]:
unmatched_test_predict['prediction'] = unmatched_test_predict


In [0]:

df_not_matched['category']=unmatched_test_predict

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [0]:
predictions = df_matched.append(df_not_matched,ignore_index=True) 
predictions = pd.merge(fashion_test, predictions,on="itemid", how='inner')
predictions.to_csv("fashion_stringmatch_dnn_7.csv")

from google.colab import files
files.download("/content/drive/My Drive/Colab Notebooks/fashion_stringmatch_dnn_7.csv")

In [0]:
!ls

 2_df_matched.csv
 2_df_not_matched.csv
 3_df_matched.csv
 3_df_not_matched.csv
'75% fashion (2).ipynb'
 combined_v1.ipynb
'Copy of Text Classifier(Bag of words).ipynb'
'Copy of xy_test'
 df_matched.csv
 df_not_matched.csv
'fashion_stringmatch_dnn (1).gsheet'
 fashion_stringmatch_dnn_3.csv
 fashion_stringmatch_dnn.csv
 fashion_stringmatch_dnn.gsheet
'preprocessing test.ipynb'
 test.ipynb
 Untitled0.ipynb
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 Untitled4.ipynb
 xy_common_words_fashion.ipynb
 xy_stringmatch_nomatch_split.ipynb
 xy_test.ipynb
'(YeeHan) Accessing Data from Google Drive.ipynb'


## THE END

In [0]:
title1 = "focallure kaos t shirt longgar cold shoulder lengan panjang warna polos untuk wanita"
title1 = "women casual o neck long sleeve lace patchwork a line pleated sexy dress"

In [0]:
predicted1 = process.extract(title1,fashion_category_titles, scorer=fuzz.ratio)
print(predicted1)
print(len(predicted1))
print(predicted1[0][0])

[('Shirt', 38, 27), ('Tshirt', 37, 25), ('Party Dress', 31, 19), ('Casual Dress', 30, 18), ('A Line Dress', 30, 21)]
5
Shirt


In [0]:
predicted1 = process.extract(title1,fashion_category_titles, scorer=fuzz.partial_ratio)
print(predicted1)

# print(predicted1[2])
threshold=99

low =[]

print(len(predicted1))
for item in predicted1:
  if item[1]<threshold:
    low.append(item)

for item in low:
  predicted1.remove(item)
print(predicted1)

[('maxy dress', 80, -20), ('Maxi Dress', 70, 20), ('Line Dress', 70, 21), ('Casual Dress', 67, 18), ('Big Size Dress', 64, 24)]
5
[]


IndexError: ignored

In [0]:
!mv df_matched.csv df_not_matched.csv /drive/My Drive/NDSC/data/

mv: target 'Drive/NDSC/data/' is not a directory


In [0]:
df_test['Category'] = prediction
df_submit = df_test[['itemid', 'Category']].copy()
df_submit.to_csv('submission.csv', index=False)

In [0]:
fashion_predictions = pd.read_csv('/content/drive/My Drive/Colab Notebooks/fashion_stringmatch_dnn.csv')

In [0]:
for index, row in fashion_predictions.iterrows():
  cat_num=row['category']
  if cat_num <17:
    new_num=cat_num+17
    fashion_predictions = fashion_predictions.replace({'category':row}, new_num)

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [0]:
fashion_predictions.to_csv("1_fashion_predictions.csv")
from google.colab import files
files.download("/content/drive/My Drive/Colab Notebooks/1_fashion_predictions.csv")